In [52]:
import pandas as pd
import numpy as np
import csv

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

root = '/Users/schwalmdaniel/github/kaggle/mnist'
#root = 'd:/dev/python/kaggle/titanic'

train=pd.read_csv(root + "/train.csv")
test=pd.read_csv(root + "/test.csv")



In [53]:
X = train.drop(['label'], axis=1)
# create our feature matrix by removing the response variable
print ("learning from {} rows".format(X.shape[0]))
y = train['label']

learning from 42000 rows


In [54]:
# Normalize the data. Each pixel comes with a value between 0-255. CNN works better with values between 0-1.
X /= 255.0
test /= 255.0

In [55]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X = X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)


In [56]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
y = to_categorical(y, num_classes = 10)

In [57]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),activation ='relu',input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),activation ='relu'))
model.add(Flatten())
model.add(Dense(10, activation = "softmax"))

model.compile(Adam(lr=.0001), loss = "categorical_crossentropy", metrics=["accuracy"])

#history = model.fit(X, y, batch_size = 100, epochs = 10, validation_split=0.1,verbose = 2)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.1)

datagen.fit(X_train)



history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=100),epochs = 15, validation_data=(X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // 100)

Epoch 1/15
 - 109s - loss: 1.0020 - acc: 0.6910 - val_loss: 0.2246 - val_acc: 0.9433
Epoch 2/15
 - 98s - loss: 0.3605 - acc: 0.8965 - val_loss: 0.1580 - val_acc: 0.9579
Epoch 3/15
 - 91s - loss: 0.2798 - acc: 0.9176 - val_loss: 0.1352 - val_acc: 0.9633
Epoch 4/15
 - 91s - loss: 0.2458 - acc: 0.9289 - val_loss: 0.1153 - val_acc: 0.9700
Epoch 5/15
 - 91s - loss: 0.2176 - acc: 0.9377 - val_loss: 0.1100 - val_acc: 0.9712
Epoch 6/15
 - 90s - loss: 0.2025 - acc: 0.9419 - val_loss: 0.1017 - val_acc: 0.9724
Epoch 7/15
 - 91s - loss: 0.1802 - acc: 0.9483 - val_loss: 0.0972 - val_acc: 0.9745
Epoch 8/15
 - 91s - loss: 0.1725 - acc: 0.9513 - val_loss: 0.0884 - val_acc: 0.9748
Epoch 9/15
 - 881s - loss: 0.1586 - acc: 0.9556 - val_loss: 0.0928 - val_acc: 0.9738
Epoch 10/15
 - 97s - loss: 0.1518 - acc: 0.9564 - val_loss: 0.0807 - val_acc: 0.9776
Epoch 11/15
 - 92s - loss: 0.1444 - acc: 0.9582 - val_loss: 0.0867 - val_acc: 0.9738
Epoch 12/15
 - 91s - loss: 0.1377 - acc: 0.9595 - val_loss: 0.0795 - val

In [58]:
preds = model.predict_classes(test,batch_size=100,verbose=2)

In [59]:
test=pd.read_csv(root + "/test.csv")

predicted = pd.DataFrame()
predicted['ImageId'] = test.index + 1
predicted['Label'] = preds
predicted[['ImageId', 'Label']] = predicted[['ImageId', 'Label']].astype(int)
predicted.to_csv(root + '/submission_keras.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)

predicted.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
